In [ ]:
# Copyright 2025 DeepMind Technologies Limited. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-gemini/genai-processors/blob/main/notebooks/function_calling.ipynb)

# Function Calling with Processors 📞

The GenAI Processor library provides a processor to run Function Calls with any
model that can produce GenAI FunctionCall parts. This notebook provides a
step-by-step guide on how to do this.



## 1. 🛠️ Setup

First, install the GenAI Processors library:

In [ ]:
!pip install genai-processors

### API Key

You will need an API key to call the Genai models in this notebook. If you have
not done so already, obtain your API key from Google AI Studio, and import it as
a secret in Colab (recommended) or directly set it below.

In [ ]:
from google.colab import userdata

API_KEY = userdata.get('GOOGLE_API_KEY')


## 2. 👆 Define the functions to be called

The Tool definition will be automatically derived from the function signature.
So it is advised to have a good docstring that covers the arguments and the
returned value. We use introspection though and the model will see the function
schema even if docstring is missing.

In [ ]:
def get_weather(location: str) -> str:
  """Returns the weather information.

  The weather information will contain the temperature in Celsius.

  Args:
    location: name of the city, region or place where the weather is requested.

  Returns:
    a string describing the weather at the provided location.
  """
  return f'Weather in {location} is 21 degrees celcius and rainy'


def to_fahrenheit(celsius: int) -> str:
  """Returns fahrenheit temperature given the value in celsius."""
  fahrenheit = (celsius * 9 / 5) + 32
  return fahrenheit

## 3. ✨ Create a function calling processor

We will use here the Gemini API for our model. We first configure the model to
output function calls for our two python functions.

If you choose other models, you will need to configure them to make sure they
can output function calls.

**NOTE**: we need to disable the automatic function calling that implements
another function calling inside the Genai SDK to avoid duplicate function calls.
Using this processor-based function calling is needed for other services that do
not offer automatic function calling. It also opens up the mechanics of function
calling for ad-hoc development (e.g. different ways of handling errors).

In [ ]:
from genai_processors.core import genai_model
from google.genai import types as genai_types

# Initialize the GenAI model processor
# Replace 'gemini-2.5-flash' with your desired model name
genai_processor = genai_model.GenaiModel(
    api_key=API_KEY,
    model_name="gemini-2.5-flash",
    generate_content_config=genai_types.GenerateContentConfig(
        tools=[get_weather, to_fahrenheit],
        automatic_function_calling=genai_types.AutomaticFunctionCallingConfig(
            disable=True
        ),
    ),
)

Then adding function calling is done as follows:

In [ ]:
from genai_processors.core import function_calling

fc = function_calling.FunctionCalling(
    genai_processor,
    fns=[get_weather, to_fahrenheit],
)

## 4. ▶️ Run the function calling processor

The function calling processor is typically used on a stream of user prompts.

In [ ]:
from genai_processors import streams
import nest_asyncio

nest_asyncio.apply()  # Needed to run async loops in Colab

input_stream = streams.stream_content(
    ['What is the weather in Cherbourg? Give the temperature in Fahrenheit.']
)

async for part in fc(input_stream):
  if not part.substream_name:
    # default substream - contains what the user would see.
    print(f'\033[0m {part.text}', flush=True, end='')

  if part.substream_name:
    # subtream_name = "function_call" / internal function calls.
    if part.function_call:
      print(
          f'\033[96m FC: {part.function_call.name}: {part.function_call.args}',
          flush=True,
      )
    elif part.function_response:
      print(
          f'\033[96m FR: {part.function_response.response}',
          flush=True,
      )